In [ ]:
%load_ext autoreload
%autoreload 2

# Behavior Data Interface

In [ ]:
from shuler_lab_to_nwb.behavior_datainterface import ShulerBehaviorInterface
from datetime import datetime
from dateutil import tz

In [ ]:
# Source data files
trials_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/data_2022-09-14_12-13-31.csv"
reference_ts_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/ES029_2022-09-14_bot72_0_g0_tcat.imec0.ap.SY_384_6_0.txt"

# Instantiate data interface
data_interface = ShulerBehaviorInterface(
    trials_file_path=trials_file,
    reference_timestamps_file_path=reference_ts_file
)

# Edit metadata
metadata = data_interface.get_metadata()
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata['NWBFile'].update(session_start_time=session_start_time)

# Run conversion
data_interface.run_conversion(
    nwbfile_path="quicktest.nwb",
    metadata=metadata,
    overwrite=True
)

In [ ]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO('quicktest.nwb', mode='r')
nwbfile = io.read()

nwb2widget(nwbfile)

# Full conversion: SpikeGLX + Behavior

In [ ]:
from shuler_lab_to_nwb.nwbconverter import ShulerNWBConverter
import json

In [ ]:
# Optionally, you can also create a converter class that inherits from NWBConverter
# Using custom converter classes allows you to combine any number of data interfaces
from neuroconv import NWBConverter
from neuroconv.datainterfaces import SpikeGLXRecordingInterface
from shuler_lab_to_nwb.behavior_datainterface import ShulerBehaviorInterface

class ShulerNWBConverter(NWBConverter):
    data_interface_classes = dict(
        Recording=SpikeGLXRecordingInterface,
        Behavior=ShulerBehaviorInterface,
    )

In [ ]:
# Source data files
spikeglx_file_path="/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/ES029_2022-09-14_bot72_0_g0_imec0/ES029_2022-09-14_bot72_0_g0_t0.imec0.ap.bin"
subject_metadata_file_path="/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/subject_metadata.json"
trials_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/data_2022-09-14_12-13-31.csv"
reference_ts_file = "/media/luiz/storage2/taufferconsulting/client_catalystneuro/project_shuler/ES029_2022-09-14_bot72_0_g0/ES029_2022-09-14_bot72_0_g0_tcat.imec0.ap.SY_384_6_0.txt"

source_data = dict(
    Recording=dict(
        file_path=spikeglx_file_path,
    ),
    Behavior=dict(
        trials_file_path=trials_file,
        reference_timestamps_file_path=reference_ts_file
    )
)

# Instantiate converter
converter = ShulerNWBConverter(source_data=source_data)

# Edit metadata
metadata = converter.get_metadata()
with open(subject_metadata_file_path, 'r') as f:
    subject_metadata = json.load(f)
metadata['Subject'].update(subject_metadata)

# Run conversion
converter.run_conversion(
    nwbfile_path="quicktest_full.nwb",
    metadata=metadata,
    overwrite=True
)


In [ ]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO('quicktest_full.nwb', mode='r')
nwbfile = io.read()

nwb2widget(nwbfile)